In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load Data
try:
    df = pd.read_csv('../Alternative CPA Pathways Survey_December 31, 2025_09.45.csv')
except FileNotFoundError:
    print("Error: File not found.")
    df = None

if df is not None:
    # --- Data Slicing ---
    # Row 0: Headers (e.g., Q29)
    # Row 1: Question Text (e.g., "How likely...") -> This corresponds to df.iloc[0]
    # Row 2: Import IDs (e.g., {"ImportId":"QID29"}) -> This corresponds to df.iloc[1]
    # Row 3+: Data -> This corresponds to df.iloc[2:]
    
    # We slice from Row 2 onwards (index 1 in 0-based indexing after header), 
    # effectively skipping the Question Text row.
    df_data = df.iloc[1:].copy()

    # --- Filtering ---
    # We filter for Undergraduates and Graduates based on Q27.
    # Note: The Import ID row (if present) will have Q27='{"ImportId":"QID27"}' 
    # and will be filtered out automatically.
    
    undergrads = df_data[df_data['Q27'] == 'Undergraduate'].copy()
    grads = df_data[df_data['Q27'] == 'Graduate'].copy()
    
    # Print data counts
    print(f"Total rows after slicing: {len(df_data)}")
    print(f"Undergraduates: {len(undergrads)}")
    print(f"Graduates: {len(grads)}")

    # --- Task A: Cannibalization (Undergrads) ---
    # Map Q29
    q29_map = {
        'Very unlikely': 1,
        'Somewhat unlikely': 2,
        'Neither likely nor unlikely': 3,
        'Somewhat likely': 4,
        'Very likely': 5
    }
    undergrads['Q29_num'] = undergrads['Q29'].map(q29_map)

    # Map Q52
    q52_map = {
        'Significantly decreased desire': -2,
        'Decreased desire': -1,
        'No change in desire': 0,
        'Increased desire': 1,
        'Significantly increased desire': 2
    }
    undergrads['Q52_num'] = undergrads['Q52'].map(q52_map)

    # Calculate Distribution
    if not undergrads.empty:
        dist = undergrads.groupby('Q29_num')['Q52_num'].value_counts(normalize=True).unstack(fill_value=0)
        # Sort columns by logical order: -2 to 2
        cols_order = [-2, -1, 0, 1, 2]
        # Ensure all columns exist
        for col in cols_order:
            if col not in dist.columns:
                dist[col] = 0
        dist = dist[cols_order]

        # Plot 100% Stacked Bar Chart
        ax = dist.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='coolwarm_r')
        plt.title('Impact on Desire by CPA Likelihood (Undergraduates)')
        plt.xlabel('CPA Likelihood (1=Very Unlikely, 5=Very Likely)')
        plt.ylabel('Percentage of Respondents')
        plt.legend(title='Impact on Desire (-2 to +2)', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig('../outputs/task_a_cannibalization.png')
        plt.show()

        # Calculate Cannibalization Rate
        # % of "Very Likely" (5) CPA students who selected "Decreased" (-1) or "Significantly Decreased" (-2) desire.
        cannibalization_rate = 0
        if 5 in dist.index:
            cannibalization_rate = (dist.loc[5, -2] + dist.loc[5, -1]) * 100
            
        print(f"Cannibalization Rate (Very Likely CPA Students): {cannibalization_rate:.2f}%")
    else:
        print("No undergraduate data found.")

    # --- Task B: Value Proposition (Grads) ---
    if not grads.empty:
        # Columns Q24_1 to Q24_6
        q24_cols = ['Q24_1', 'Q24_2', 'Q24_3', 'Q24_4', 'Q24_5', 'Q24_6']
        # Convert to numeric
        for col in q24_cols:
            grads[col] = pd.to_numeric(grads[col], errors='coerce')

        # Calculate Mean Rank
        mean_ranks = grads[q24_cols].mean()

        # Map labels
        q24_labels = {
            'Q24_1': 'CPA Exam Prep',
            'Q24_2': 'Networking',
            'Q24_3': 'Faculty Interaction',
            'Q24_4': 'Technical Skills',
            'Q24_5': 'Soft Skills',
            'Q24_6': 'Recruiting/Internships'
        }
        mean_ranks.index = mean_ranks.index.map(q24_labels)

        # Sort (Ascending: Lower is better/harder to replace)
        mean_ranks_sorted = mean_ranks.sort_values(ascending=True)

        # Plot Horizontal Bar Chart
        mean_ranks_plotting = mean_ranks_sorted.iloc[::-1]

        plt.figure(figsize=(10, 6))
        mean_ranks_plotting.plot(kind='barh', color='skyblue')
        plt.title('Graduate Student Perception: Value of Degree Elements')
        plt.xlabel('Mean Rank (Lower Score = Harder to Replace)')
        plt.grid(axis='x', linestyle='--', alpha=0.7)
        
        # Add annotation
        plt.text(0.5, -0.15, "Lower Score = Harder to Replace", transform=plt.gca().transAxes, ha='center', fontsize=10, style='italic')
        
        plt.tight_layout()
        plt.savefig('../outputs/task_b_value_proposition.png')
        plt.show()

        # Top 2 "Hardest to Replace"
        top_2 = mean_ranks_sorted.head(2)
        print("Top 2 Hardest to Replace:")
        for item, rank in top_2.items():
            print(f"- {item}: {rank:.2f}")
    else:
        print("No graduate data found.")


# Summary of Findings

## Task A: The "Cannibalization" Risk
*   **Cannibalization Rate:** 37.50%
    *   This represents the percentage of "Very Likely" CPA students who indicated a "Decreased" or "Significantly Decreased" desire to pursue the graduate degree.

## Task B: The "Value Proposition"
*   **Top 2 "Hardest to Replace" Aspects:**
    1.  **CPA Exam Prep** (Mean Rank: 2.70)
    2.  **Technical Skills** (Mean Rank: 2.79)

---
*Data represents a point-in-time snapshot of student sentiment (Nov-Dec 2025).*
